
\begin{center}
Chun-Yuan (Scott) Chiu
\end{center}
\begin{center}
chunyuac@andrew.cmu.edu
\end{center}

# 1. {-}

In [33]:
from pandas import DataFrame
from scipy.special import beta
from scipy.optimize import minimize, root_scalar
from scipy.integrate import quad
from scipy import stats
import numpy as np
import numdifftools as nd

def pdf(x, mu, k, n, lam, sigma2, log=False):
    '''
    k > 0, n > 2, sigma2 > 0, -1 < lam < 1
    '''
    sigma = np.sqrt(sigma2)

    S = np.sqrt(1 + 3*(lam**2) - 4*(lam**2)*((beta(2/k, (n-1)/k)**2)/(beta(1/k, n/k)*beta(3/k, (n-2)/k))))
    c = 0.5*k*np.sqrt((beta(1/k, n/k)**(-3))*beta(3/k, (n-2)/k))*S/sigma
    theta = ((k/(n-2))**(1/k))*np.sqrt(beta(1/k, n/k)/beta(3/k, (n-2)/k))/S
    
    if log:
        return np.log(c)-(n+1)*np.log(1 + (k/(n-2))*(np.abs(x-mu)/(sigma*theta*(1 + np.sign(x-mu)*lam)))**k)/k
    else:
        return c*(1+(k/(n-2))*(np.abs(x-mu)/(sigma*theta*(1 + np.sign(x-mu)*lam)))**k)**(-(n+1)/k)

def cdf(x, mu, k, n, lam, sigma2):
    integrand = lambda u: pdf(u-mu, 0, k, n, lam, sigma2)
    return quad(integrand, 0, x)[0] + (1-lam)/2

def ppf(y, mu, k, n, lam, sigma2):
    assert 0 < y < 1, 'y must be in [0, 1].'  
    res = root_scalar(lambda x: cdf(x, mu, k, n, lam, sigma2)-y, method='newton', x0=mu, fprime=lambda x: pdf(x, mu, k, n, lam, sigma2))
    assert res.converged, "Newton's method failed to converge."
    return res.root

def rvs(size, mu, k, n, lam, sigma2):
    return np.array([ppf(u, mu, k, n, lam, sigma2) for u in np.random.uniform(size=size)])

def fitSkewT(x, allowshift=False):
    
    if(allowshift):
        initpars = [np.mean(x), 2, 15, 2*(np.array(x)>0).sum()/len(x)-1, np.var(x)]
        def negloglikelihood(params, x):
            mu, k, n, lam, sigma2 = params
            return (-1)*pdf(x, mu, k, n, lam, sigma2, log=True).sum()
        
        mleout = minimize(negloglikelihood, initpars, args=(x), method='L-BFGS-B', bounds=[(None, None), (0.001, None), (2.001, None), (-0.999, 0.999), (0.001, None)])
        hessfunc = nd.Hessian(negloglikelihood)
        return mleout.x, np.linalg.inv(hessfunc(mleout.x, x))
    
    else:
        initpars = [2, 15, 2*(x>0).sum()/len(x)-1, np.var(x)]
        def negloglikelihood(params, x):
            k, n, lam, sigma2 = params
            return (-1)*pdf(x, 0, k, n, lam, sigma2, log=True).sum()
        
        mleout = minimize(negloglikelihood, initpars, args=(x), method='L-BFGS-B', bounds=[(0.001, None), (2.001, None), (-0.999, 0.999), (0.001, None)])
        hessfunc = nd.Hessian(negloglikelihood)
        mat = hessfunc(mleout.x, x)
        return mleout.x, np.linalg.inv(hessfunc(mleout.x, x))

size = 10000
k = 2
n = 10
lam = -0.5
sigma2 = 1
mu = 0

x = rvs(size, mu, k, n, lam, sigma2)

mle, var = fitSkewT(x, allowshift=False)
z = stats.norm.ppf(1-0.025)

print('MLE for k, n, lambda and sigma2: ', mle)
print()
print('Covariance matrix of the MLE: \n', DataFrame(var, columns=['k', 'n', 'lambda', 'sigma2'], index=['k', 'n', 'lambda', 'sigma2']))
print()
print('Confidence Intervals for the 4 parameters: ', [(mle[i] - z*np.sqrt(var[i][i]), mle[i] + z*np.sqrt(var[i][i])) for i in range(4)])


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in log
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in sqrt
  del sys.path[0]
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in sqrt


MLE for k, n, lambda and sigma2:  [ 2.07735094  9.10214337 -0.50524015  0.9744172 ]

Covariance matrix of the MLE: 
                    k         n        lambda    sigma2
k       1.102641e-02 -0.135026  6.534968e-07  0.000026
n      -1.350260e-01  2.202621  5.291868e-05 -0.007730
lambda  6.534968e-07  0.000053  2.924823e-05 -0.000016
sigma2  2.550303e-05 -0.007730 -1.559490e-05  0.000352

Confidence Intervals for the 4 parameters:  [(1.8715416006885628, 2.283160276142435), (6.193316012424198, 12.010970717973926), (-0.5158399570026023, -0.49464034785258126), (0.9376290134534304, 1.011205393442958)]
